In [ ]:
import os.path
from time import perf_counter

import numpy as np
import scipy.linalg as la
import matplotlib
import matplotlib.pyplot as plt
import netCDF4 as nc4
import matplotlib.animation as animation

import bin_model as bm

# Plotting an example output

In [ ]:
CTENS_FILE_NAME = "collision_data/Hall_ScottChen_ctens_nb336.nc"
EXPERIMENT_FILE_NAME = \
    "convergence_experiments/mass_convergence_experiment_nb336.nc"

In [ ]:
with nc4.Dataset(CTENS_FILE_NAME, "r") as nc:
    netcdf_file = bm.NetcdfFile(nc)
    _, ckern, _, ctens = netcdf_file.read_ckgt()
with nc4.Dataset(EXPERIMENT_FILE_NAME, "r") as nc:
    netcdf_file = bm.NetcdfFile(nc)
    exp = netcdf_file.read_full_experiment([ctens])

In [ ]:
nt = exp.num_time_steps
nb = exp.mass_grid.num_bins
bb = exp.mass_grid.bin_bounds
times = exp.times
dsds = np.zeros((nt, nb))
for i in range(nt):
    dsds[i,:] = exp.states[i].dsd() / exp.mass_grid.bin_widths
plt.pcolor(bb, times, np.log10(np.maximum(dsds[1:,:], 1.e-9)), cmap='jet')
plt.colorbar()

In [ ]:
const = exp.constants
grid = exp.mass_grid
ymin = 0.
ymax = 3.
fig = plt.figure(figsize=(5, 4))
ax = fig.add_subplot(autoscale_on=False,
                     xlim=(2. + grid.lx_min/(3.*np.log(10)),
                           2. + grid.lx_max/(3.*np.log(10))),
                     ylim=(ymin, ymax))
ax.set_xlabel("$log_{10}(D)$ ($D$ in microns)")
ax.set_ylabel("$dm/dlog_{10}(D)$ (g/kg)")
ax.grid()

plot_lds = 2. + grid.bin_bounds[:-1] / (3.*np.log(10))
line, = ax.plot(plot_lds, exp.states[0].dsd(),
                'o-', lw=2)
time_template = 'time = %.1fs'
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
plt.vlines(np.log10(const.rain_d)+6., ymin, ymax, 'k')

mass_convert = 1.e3 * (3.*np.log(10))
frame_stride = 1
def animate(i):
    ti = frame_stride * i
    thisy = mass_convert * exp.states[ti].dsd()
    line.set_data(plot_lds, thisy)
    time_text.set_text(time_template % exp.times[ti])
    return line, time_text

num_steps = exp.num_time_steps
num_frames = num_steps // frame_stride
print(num_steps)
ani = animation.FuncAnimation(
    fig, animate, num_frames,
    interval=6000. / num_frames,
    blit=True)
ani.save("mass_evolution.gif")
plt.show()

# Old plot code

In [ ]:
D = 18
NU = 10

FIXED_MOMENTS = [0, 3, 9]
CHECKED_MOMENTS = [1, 2, 4, 5, 7, 8]

TEMPLATE_1CAT_4MOM = os.path.join(
    "sensitivity_experiments", "experiment_d{:02d}nu{:02d}_f0-3-6-9.nc")

TEMPLATE_1CAT_4MOM_MULTIMOM = os.path.join(
    "sensitivity_experiments", "experiment_d{:02d}nu{:02d}_f{}-{}-{}-{}.nc")

TEMPLATE_1CAT_5MOM = os.path.join(
    "sensitivity_experiments", "experiment_d{:02d}nu{:02d}_f0-3-6-9-12.nc")

TEMPLATE_1CAT_6MOM = os.path.join(
    "sensitivity_experiments", "experiment_d{:02d}nu{:02d}_f0-3-6-9-12-15.nc")

TEMPLATE_2CAT_2MOM = os.path.join(
    "sensitivity_experiments", "experiment_d{:02d}nu{:02d}_c0-3_r0-3.nc")

TEMPLATE_2CAT_2P5MOM = os.path.join(
    "sensitivity_experiments", "experiment_d{:02d}nu{:02d}_c0-3-6_r0-3.nc")

TEMPLATE_2CAT_3MOM = os.path.join(
    "sensitivity_experiments", "experiment_d{:02d}nu{:02d}_c0-3-6_r0-3-6.nc")

TEMPLATE_1CAT_4MOM_NM = os.path.join(
    "sensitivity_experiments", "experiment_d{:02d}nu{:02d}_f-3-0-3-6.nc")

TEMPLATE_1CAT_5MOM_NM = os.path.join(
    "sensitivity_experiments", "experiment_d{:02d}nu{:02d}_f-3-0-3-6-9.nc")

TEMPLATE_1CAT_6MOM_NM = os.path.join(
    "sensitivity_experiments", "experiment_d{:02d}nu{:02d}_f-6--3-0-3-6-9.nc")

In [ ]:
FILE_NAME = TEMPLATE_1CAT_4MOM.format(D, NU)
with nc4.Dataset(FILE_NAME, "r") as nc:
    netcdf_file = bm.NetcdfFile(nc)
    exp_1cat_4mom = netcdf_file.read_full_experiment([ctens])

exp_1cat_4mom_multimom = dict()
for check_mom in CHECKED_MOMENTS:
    moments = sorted(FIXED_MOMENTS + [check_mom])
    FILE_NAME = TEMPLATE_1CAT_4MOM_MULTIMOM.format(D, NU,
                                                   moments[0], moments[1],
                                                   moments[2], moments[3])
    with nc4.Dataset(FILE_NAME, "r") as nc:
        netcdf_file = bm.NetcdfFile(nc)
        exp_1cat_4mom_multimom[check_mom] = netcdf_file.read_full_experiment([ctens])

FILE_NAME = TEMPLATE_1CAT_5MOM.format(D, NU)
with nc4.Dataset(FILE_NAME, "r") as nc:
    netcdf_file = bm.NetcdfFile(nc)
    exp_1cat_5mom = netcdf_file.read_full_experiment([ctens])

FILE_NAME = TEMPLATE_1CAT_6MOM.format(D, NU)
with nc4.Dataset(FILE_NAME, "r") as nc:
    netcdf_file = bm.NetcdfFile(nc)
    exp_1cat_6mom = netcdf_file.read_full_experiment([ctens])

FILE_NAME = TEMPLATE_2CAT_2MOM.format(D, NU)
with nc4.Dataset(FILE_NAME, "r") as nc:
    netcdf_file = bm.NetcdfFile(nc)
    exp_2cat_2mom = netcdf_file.read_full_experiment([ctens])

FILE_NAME = TEMPLATE_2CAT_2P5MOM.format(D, NU)
with nc4.Dataset(FILE_NAME, "r") as nc:
    netcdf_file = bm.NetcdfFile(nc)
    exp_2cat_2p5mom = netcdf_file.read_full_experiment([ctens])

FILE_NAME = TEMPLATE_2CAT_3MOM.format(D, NU)
with nc4.Dataset(FILE_NAME, "r") as nc:
    netcdf_file = bm.NetcdfFile(nc)
    exp_2cat_3mom = netcdf_file.read_full_experiment([ctens])

FILE_NAME = TEMPLATE_1CAT_4MOM_NM.format(D, NU)
with nc4.Dataset(FILE_NAME, "r") as nc:
    netcdf_file = bm.NetcdfFile(nc)
    exp_1cat_4mom_nm = netcdf_file.read_full_experiment([ctens])

FILE_NAME = TEMPLATE_1CAT_5MOM_NM.format(D, NU)
with nc4.Dataset(FILE_NAME, "r") as nc:
    netcdf_file = bm.NetcdfFile(nc)
    exp_1cat_5mom_nm = netcdf_file.read_full_experiment([ctens])

FILE_NAME = TEMPLATE_1CAT_6MOM_NM.format(D, NU)
with nc4.Dataset(FILE_NAME, "r") as nc:
    netcdf_file = bm.NetcdfFile(nc)
    exp_1cat_6mom_nm = netcdf_file.read_full_experiment([ctens])

In [ ]:
#num_time_steps = exp_1cat_4mom.num_time_steps
#autos = np.zeros((num_time_steps+1,))
#accrs = np.zeros((num_time_steps+1,))
#cloud_lx = np.log(const.rain_m)
#cloud_idx = grid.find_bin(cloud_lx)
#if (grid.bin_bounds[cloud_idx+1] - cloud_lx) < 1.e-10:
#    cloud_idx += 1
#cloud_vector = np.zeros((nb,))
#cloud_vector[:cloud_idx] = 1.
#for i in range(num_time_steps+1):
#    print(i)
#    autos[i], accrs[i] = \
#        exp_1cat_4mom.states[i].rain_prod_breakdown(ctens, cloud_vector)

In [ ]:
num_time_steps = exp_1cat_4mom.num_time_steps

In [ ]:
moments = [0, 3, 6, 9]
nmom = len(moments)
sigma_diags = np.zeros((num_time_steps, nmom))
for i in range(num_time_steps):
    perturb_cov = exp_1cat_4mom.states[i].perturb_cov()
    for j in range(nmom):
        sigma_diags[i,j] = np.sqrt(perturb_cov[j,j])

In [ ]:
fig = plt.figure(figsize=(5, 4))
for n in range(nmom):
    plt.plot(exp_1cat_4mom.times, sigma_diags[:,n],
             label="M{}".format(moments[n]))
ymax = 10.
#for i in range(num_time_steps+1):
#    if autos[i] < accrs[i]:
#        time = t_eval[i]
#        break
#plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
#for i in range(num_time_steps+1):
#    if autos[i] > 0.01 * autos.max():
#        time = t_eval[i]
#        break
#plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
moments = [0, 3, 6, 9, 12]
nmom = len(moments)
sigma_diags = np.zeros((num_time_steps, nmom))
for i in range(num_time_steps):
    perturb_cov = exp_1cat_5mom.states[i].perturb_cov()
    for j in range(nmom):
        sigma_diags[i,j] = np.sqrt(perturb_cov[j,j])

In [ ]:
fig = plt.figure(figsize=(5, 4))
for n in range(nmom):
    plt.plot(exp_1cat_5mom.times, sigma_diags[:,n],
             label="M{}".format(moments[n]))
ymax = 10.
#for i in range(num_time_steps+1):
#    if autos[i] < accrs[i]:
#        time = t_eval[i]
#        break
#plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
#for i in range(num_time_steps+1):
#    if autos[i] > 0.01 * autos.max():
#        time = t_eval[i]
#        break
#plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
moments = [0, 3, 6, 9, 12, 15]
nmom = len(moments)
sigma_diags = np.zeros((num_time_steps, nmom))
for i in range(num_time_steps):
    perturb_cov = exp_1cat_6mom.states[i].perturb_cov()
    for j in range(nmom):
        sigma_diags[i,j] = np.sqrt(perturb_cov[j,j])

In [ ]:
fig = plt.figure(figsize=(5, 4))
for n in range(nmom):
    plt.plot(exp_1cat_6mom.times, sigma_diags[:,n],
             label="M{}".format(moments[n]))
ymax = 10.
#for i in range(num_time_steps+1):
#    if autos[i] < accrs[i]:
#        time = t_eval[i]
#        break
#plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
#for i in range(num_time_steps+1):
#    if autos[i] > 0.01 * autos.max():
#        time = t_eval[i]
#        break
#plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
nmom = 4
sigma_diags = np.zeros((num_time_steps, nmom))
for i in range(num_time_steps):
    perturb_cov = exp_2cat_2mom.states[i].perturb_cov()
    for j in range(nmom):
        sigma_diags[i,j] = np.sqrt(perturb_cov[j,j])

In [ ]:
fig = plt.figure(figsize=(5, 4))
labels = ["Cloud M0", "Cloud M3", "Rain M0", "Rain M3"]
for n in range(nmom):
    plt.plot(exp_2cat_2mom.times, sigma_diags[:,n], label=labels[n])
ymax = 10.
#for i in range(num_time_steps+1):
#    if autos[i] < accrs[i]:
#        time = t_eval[i]
#        break
#plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
#for i in range(num_time_steps+1):
#    if autos[i] > 0.01 * autos.max():
#        time = t_eval[i]
#        break
#plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
nmom = 5
sigma_diags = np.zeros((num_time_steps, nmom))
for i in range(num_time_steps):
    perturb_cov = exp_2cat_2p5mom.states[i].perturb_cov()
    for j in range(nmom):
        sigma_diags[i,j] = np.sqrt(perturb_cov[j,j])

In [ ]:
fig = plt.figure(figsize=(5, 4))
labels = ["Cloud M0", "Cloud M3", "Cloud M6", "Rain M0", "Rain M3"]
for n in range(nmom):
    plt.plot(exp_2cat_2p5mom.times, sigma_diags[:,n], label=labels[n])
ymax = 10.
#for i in range(num_time_steps+1):
#    if autos[i] < accrs[i]:
#        time = t_eval[i]
#        break
#plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
#for i in range(num_time_steps+1):
#    if autos[i] > 0.01 * autos.max():
#        time = t_eval[i]
#        break
#plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
nmom = 6
sigma_diags = np.zeros((num_time_steps, nmom))
for i in range(num_time_steps):
    perturb_cov = exp_2cat_3mom.states[i].perturb_cov()
    for j in range(nmom):
        sigma_diags[i,j] = np.sqrt(perturb_cov[j,j])

In [ ]:
fig = plt.figure(figsize=(5, 4))
labels = ["Cloud M0", "Cloud M3", "Cloud M6",
          "Rain M0", "Rain M3", "Rain M6"]
for n in range(nmom):
    plt.plot(exp_2cat_3mom.times, sigma_diags[:,n], label=labels[n])
ymax = 10.
#for i in range(num_time_steps+1):
#    if autos[i] < accrs[i]:
#        time = t_eval[i]
#        break
#plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
#for i in range(num_time_steps+1):
#    if autos[i] > 0.01 * autos.max():
#        time = t_eval[i]
#        break
#plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
moments = [-3, 0, 3, 6]
nmom = len(moments)
sigma_diags = np.zeros((num_time_steps, nmom))
for i in range(num_time_steps):
    perturb_cov = exp_1cat_4mom_nm.states[i].perturb_cov()
    for j in range(nmom):
        sigma_diags[i,j] = np.sqrt(perturb_cov[j,j])

In [ ]:
fig = plt.figure(figsize=(5, 4))
for n in range(nmom):
    plt.plot(exp_1cat_4mom_nm.times, sigma_diags[:,n],
             label="M{}".format(moments[n]))
ymax = 10.
#for i in range(num_time_steps+1):
#    if autos[i] < accrs[i]:
#        time = t_eval[i]
#        break
#plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
#for i in range(num_time_steps+1):
#    if autos[i] > 0.01 * autos.max():
#        time = t_eval[i]
#        break
#plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
moments = [-3, 0, 3, 6, 9]
nmom = len(moments)
sigma_diags = np.zeros((num_time_steps, nmom))
for i in range(num_time_steps):
    perturb_cov = exp_1cat_5mom_nm.states[i].perturb_cov()
    for j in range(nmom):
        sigma_diags[i,j] = np.sqrt(perturb_cov[j,j])

In [ ]:
fig = plt.figure(figsize=(5, 4))
for n in range(nmom):
    plt.plot(exp_1cat_5mom_nm.times, sigma_diags[:,n],
             label="M{}".format(moments[n]))
ymax = 10.
#for i in range(num_time_steps+1):
#    if autos[i] < accrs[i]:
#        time = t_eval[i]
#        break
#plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
#for i in range(num_time_steps+1):
#    if autos[i] > 0.01 * autos.max():
#        time = t_eval[i]
#        break
#plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
moments = [-6, -3, 0, 3, 6, 9]
nmom = len(moments)
sigma_diags = np.zeros((num_time_steps, nmom))
for i in range(num_time_steps):
    perturb_cov = exp_1cat_6mom_nm.states[i].perturb_cov()
    for j in range(nmom):
        sigma_diags[i,j] = np.sqrt(perturb_cov[j,j])

In [ ]:
fig = plt.figure(figsize=(5, 4))
for n in range(nmom):
    plt.plot(exp_1cat_6mom_nm.times, sigma_diags[:,n],
             label="M{}".format(moments[n]))
ymax = 10.
#for i in range(num_time_steps+1):
#    if autos[i] < accrs[i]:
#        time = t_eval[i]
#        break
#plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
#for i in range(num_time_steps+1):
#    if autos[i] > 0.01 * autos.max():
#        time = t_eval[i]
#        break
#plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
covar_volume_1cat_4mom = np.zeros((num_time_steps,))
covar_volume_1cat_5mom = np.zeros((num_time_steps,))
covar_volume_1cat_6mom = np.zeros((num_time_steps,))
covar_volume_2cat_2mom = np.zeros((num_time_steps,))
covar_volume_2cat_2p5mom = np.zeros((num_time_steps,))
covar_volume_2cat_3mom = np.zeros((num_time_steps,))
covar_volume_1cat_4mom_nm = np.zeros((num_time_steps,))
covar_volume_1cat_5mom_nm = np.zeros((num_time_steps,))
covar_volume_1cat_6mom_nm = np.zeros((num_time_steps,))
for i in range(num_time_steps):
    perturb_cov = exp_1cat_4mom.states[i].perturb_cov()
    covar_volume_1cat_4mom[i] = np.prod(la.svdvals(perturb_cov)[:4])
    perturb_cov = exp_1cat_5mom.states[i].perturb_cov()
    covar_volume_1cat_5mom[i] = np.prod(la.svdvals(perturb_cov)[:4])
    perturb_cov = exp_1cat_6mom.states[i].perturb_cov()
    covar_volume_1cat_6mom[i] = np.prod(la.svdvals(perturb_cov)[:4])
    perturb_cov = exp_2cat_2mom.states[i].perturb_cov()
    covar_volume_2cat_2mom[i] = np.prod(la.svdvals(perturb_cov)[:4])
    perturb_cov = exp_2cat_2p5mom.states[i].perturb_cov()
    covar_volume_2cat_2p5mom[i] = np.prod(la.svdvals(perturb_cov)[:4])
    perturb_cov = exp_2cat_3mom.states[i].perturb_cov()
    covar_volume_2cat_3mom[i] = np.prod(la.svdvals(perturb_cov)[:4])
    perturb_cov = exp_1cat_4mom_nm.states[i].perturb_cov()
    covar_volume_1cat_4mom_nm[i] = np.prod(la.svdvals(perturb_cov)[:4])
    perturb_cov = exp_1cat_5mom_nm.states[i].perturb_cov()
    covar_volume_1cat_5mom_nm[i] = np.prod(la.svdvals(perturb_cov)[:4])
    perturb_cov = exp_1cat_6mom_nm.states[i].perturb_cov()
    covar_volume_1cat_6mom_nm[i] = np.prod(la.svdvals(perturb_cov)[:4])

In [ ]:
fig = plt.figure(figsize=(5, 4))
t_eval = exp_1cat_4mom.times
plt.plot(t_eval, covar_volume_1cat_4mom, label='1-cat 0-3-6-9')
plt.plot(t_eval, covar_volume_1cat_5mom, label='1-cat 0-3-6-9-12')
plt.plot(t_eval, covar_volume_1cat_6mom, label='1-cat 0-3-6-9-12-15')
plt.plot(t_eval, covar_volume_2cat_2mom, label='2-cat c0-3 r0-3')
plt.plot(t_eval, covar_volume_2cat_2p5mom, label='2-cat c0-3-6 r0-3')
plt.plot(t_eval, covar_volume_2cat_3mom, label='2-cat c0-3-6 r0-3-6')
plt.plot(t_eval, covar_volume_1cat_4mom_nm, label='1-cat -3-0-3-6')
plt.plot(t_eval, covar_volume_1cat_5mom_nm, label='1-cat -3-0-3-6-9')
plt.plot(t_eval, covar_volume_1cat_6mom_nm, label='1-cat -6--3-0-3-6-9')
#ymax = 4.5e-6
#for i in range(num_time_steps+1):
#    if autos[i] < accrs[i]:
#        time = t_eval[i]
#        break
#plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
#for i in range(num_time_steps+1):
#    if autos[i] > 0.01 * autos.max():
#        time = t_eval[i]
#        break
#plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
#plt.ylim([0., ymax])
plt.ylim([0., 100.])
plt.legend(loc='best')

In [ ]:
wvs = np.zeros((3, grid.num_bins))
wvs[0,:] = grid.moment_weight_vector(0)
wvs[1,:] = grid.moment_weight_vector(3, cloud_only=True)
wvs[2,:] = grid.moment_weight_vector(6)

In [ ]:
moms_1cat_4mom, mom_covs_1cat_4mom = \
    exp_1cat_4mom.get_moments_and_covariances(wvs)

moms_1cat_4mom_multimom = dict()
mom_covs_1cat_4mom_multimom = dict()
for check_mom in CHECKED_MOMENTS:
    moms_1cat_4mom_multimom[check_mom], \
        mom_covs_1cat_4mom_multimom[check_mom] = \
        exp_1cat_4mom_multimom[check_mom].get_moments_and_covariances(wvs)

moms_1cat_5mom, mom_covs_1cat_5mom = \
    exp_1cat_5mom.get_moments_and_covariances(wvs)
moms_1cat_6mom, mom_covs_1cat_6mom = \
    exp_1cat_6mom.get_moments_and_covariances(wvs)
moms_2cat_2mom, mom_covs_2cat_2mom = \
    exp_2cat_2mom.get_moments_and_covariances(wvs)
moms_2cat_2p5mom, mom_covs_2cat_2p5mom = \
    exp_2cat_2p5mom.get_moments_and_covariances(wvs)
moms_2cat_3mom, mom_covs_2cat_3mom = \
    exp_2cat_3mom.get_moments_and_covariances(wvs)
moms_1cat_4mom_nm, mom_covs_1cat_4mom_nm = \
    exp_1cat_4mom_nm.get_moments_and_covariances(wvs)
moms_1cat_5mom_nm, mom_covs_1cat_5mom_nm = \
    exp_1cat_5mom_nm.get_moments_and_covariances(wvs)
moms_1cat_6mom_nm, mom_covs_1cat_6mom_nm = \
    exp_1cat_6mom_nm.get_moments_and_covariances(wvs)

In [ ]:
fig = plt.figure(figsize=(5, 4))
m0_scale = const.mass_conc_scale / const.std_mass
plt.plot(t_eval, m0_scale * moms_1cat_4mom[:,0],
         'k', label='Total M0')
plt.plot(t_eval, m0_scale * np.sqrt(mom_covs_1cat_4mom[:,0,0]),
         label='1-cat 0-3-6-9')
plt.plot(t_eval, m0_scale * np.sqrt(mom_covs_1cat_5mom[:,0,0]),
         label='1-cat 0-3-6-9-12')
plt.plot(t_eval, m0_scale * np.sqrt(mom_covs_1cat_6mom[:,0,0]),
         label='1-cat 0-3-6-9-12-15')
plt.plot(t_eval, m0_scale * np.sqrt(mom_covs_2cat_2mom[:,0,0]),
         label='2-cat c0-3 r0-3')
plt.plot(t_eval, m0_scale * np.sqrt(mom_covs_2cat_2p5mom[:,0,0]),
         label='2-cat c0-3-6 r0-3')
plt.plot(t_eval, m0_scale * np.sqrt(mom_covs_2cat_3mom[:,0,0]),
         label='2-cat c0-3-6 r0-3-6')
plt.plot(t_eval, m0_scale * np.sqrt(mom_covs_1cat_4mom_nm[:,0,0]),
         label='1-cat -3-0-3-6')
plt.plot(t_eval, m0_scale * np.sqrt(mom_covs_1cat_5mom_nm[:,0,0]),
         label='1-cat -3-0-3-6-9')
plt.plot(t_eval, m0_scale * np.sqrt(mom_covs_1cat_6mom_nm[:,0,0]),
         label='1-cat -6--3-0-3-6-9')
ymax = 1.2 * m0_scale * moms_1cat_4mom[:,0].max()
#for i in range(num_time_steps+1):
#    if autos[i] < accrs[i]:
#        time = t_eval[i]
#        break
#plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
#for i in range(num_time_steps+1):
#    if autos[i] > 0.01 * autos.max():
#        time = t_eval[i]
#        break
#plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
fig = plt.figure(figsize=(5, 4))
m0_scale = const.mass_conc_scale / const.std_mass
plt.plot(t_eval, m0_scale * moms_1cat_4mom[:,0],
         'k', label='Total M0')
plt.plot(t_eval, m0_scale * np.sqrt(mom_covs_1cat_4mom[:,0,0]),
         label='1-cat 0-3-6-9')
for check_mom in CHECKED_MOMENTS:
    moments = sorted(FIXED_MOMENTS + [check_mom])
    label_str = '1-cat {}-{}-{}-{}'.format(moments[0], moments[1],
                                           moments[2], moments[3])
    plt.plot(t_eval, m0_scale
             * np.sqrt(mom_covs_1cat_4mom_multimom[check_mom][:,0,0]),
             label=label_str)
ymax = 1.2 * m0_scale * moms_1cat_4mom[:,0].max()
#for i in range(num_time_steps+1):
#    if autos[i] < accrs[i]:
#        time = t_eval[i]
#        break
#plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
#for i in range(num_time_steps+1):
#    if autos[i] > 0.01 * autos.max():
#        time = t_eval[i]
#        break
#plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
fig = plt.figure(figsize=(5, 4))
m3_scale = const.diameter_scale**3 * m0_scale
plt.plot(t_eval, m3_scale * moms_1cat_4mom[:,1], 'k',
         label='Cloud M3')
plt.plot(t_eval, m3_scale * np.sqrt(mom_covs_1cat_4mom[:,1,1]),
         label='1-cat 0-3-6-9')
plt.plot(t_eval, m3_scale * np.sqrt(mom_covs_1cat_5mom[:,1,1]),
         label='1-cat 0-3-6-9-12')
plt.plot(t_eval, m3_scale * np.sqrt(mom_covs_1cat_6mom[:,1,1]),
         label='1-cat 0-3-6-9-12-15')
plt.plot(t_eval, m3_scale * np.sqrt(mom_covs_2cat_2mom[:,1,1]),
         label='2-cat c0-3 r0-3')
plt.plot(t_eval, m3_scale * np.sqrt(mom_covs_2cat_2p5mom[:,1,1]),
         label='2-cat c0-3-6 r0-3')
plt.plot(t_eval, m3_scale * np.sqrt(mom_covs_2cat_3mom[:,1,1]),
         label='2-cat c0-3-6 r0-3-6')
plt.plot(t_eval, m3_scale * np.sqrt(mom_covs_1cat_4mom_nm[:,1,1]),
         label='1-cat -3-0-3-6')
plt.plot(t_eval, m3_scale * np.sqrt(mom_covs_1cat_5mom_nm[:,1,1]),
         label='1-cat -3-0-3-6-9')
plt.plot(t_eval, m3_scale * np.sqrt(mom_covs_1cat_6mom_nm[:,1,1]),
         label='1-cat -6--3-0-3-6-9')
ymax = 1.2 * m3_scale * moms_1cat_4mom[:,1].max()
#for i in range(num_time_steps+1):
#    if autos[i] < accrs[i]:
#        time = t_eval[i]
#        break
#plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
#for i in range(num_time_steps+1):
#    if autos[i] > 0.01 * autos.max():
#        time = t_eval[i]
#        break
#plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
fig = plt.figure(figsize=(5, 4))
m3_scale = const.diameter_scale**3 * m0_scale
plt.plot(t_eval, m3_scale * moms_1cat_4mom[:,1], 'k',
         label='Cloud M3')
plt.plot(t_eval, m3_scale * np.sqrt(mom_covs_1cat_4mom[:,1,1]),
         label='1-cat 0-3-6-9')
for check_mom in CHECKED_MOMENTS:
    moments = sorted(FIXED_MOMENTS + [check_mom])
    label_str = '1-cat {}-{}-{}-{}'.format(moments[0], moments[1],
                                           moments[2], moments[3])
    plt.plot(t_eval, m3_scale
             * np.sqrt(mom_covs_1cat_4mom_multimom[check_mom][:,1,1]),
             label=label_str)
ymax = 1.2 * m3_scale * moms_1cat_4mom[:,1].max()
#for i in range(num_time_steps+1):
#    if autos[i] < accrs[i]:
#        time = t_eval[i]
#        break
#plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
#for i in range(num_time_steps+1):
#    if autos[i] > 0.01 * autos.max():
#        time = t_eval[i]
#        break
#plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
fig = plt.figure(figsize=(5, 4))
m6_scale = const.diameter_scale**6 * m0_scale
plt.plot(t_eval, m6_scale * moms_1cat_4mom[:,2], 'k',
         label='Total M6')
plt.plot(t_eval, m6_scale * np.sqrt(mom_covs_1cat_4mom[:,2,2]),
         label='1-cat 0-3-6-9')
plt.plot(t_eval, m6_scale * np.sqrt(mom_covs_1cat_5mom[:,2,2]),
         label='1-cat 0-3-6-9-12')
plt.plot(t_eval, m6_scale * np.sqrt(mom_covs_1cat_6mom[:,2,2]),
         label='1-cat 0-3-6-9-12-15')
plt.plot(t_eval, m6_scale * np.sqrt(mom_covs_2cat_2mom[:,2,2]),
         label='2-cat c0-3 r0-3')
plt.plot(t_eval, m6_scale * np.sqrt(mom_covs_2cat_2p5mom[:,2,2]),
         label='2-cat c0-3-6 r0-3')
plt.plot(t_eval, m6_scale * np.sqrt(mom_covs_2cat_3mom[:,2,2]),
         label='2-cat c0-3-6 r0-3-6')
plt.plot(t_eval, m6_scale * np.sqrt(mom_covs_1cat_4mom_nm[:,2,2]),
         label='1-cat -3-0-3-6')
plt.plot(t_eval, m6_scale * np.sqrt(mom_covs_1cat_5mom_nm[:,2,2]),
         label='1-cat -3-0-3-6-9')
plt.plot(t_eval, m6_scale * np.sqrt(mom_covs_1cat_6mom_nm[:,2,2]),
         label='1-cat -6--3-0-3-6-9')
ymax = 1.2 * m6_scale * moms_1cat_4mom[:,2].max()
#ymax = 0.1
#for i in range(num_time_steps+1):
#    if autos[i] < accrs[i]:
#        time = t_eval[i]
#        break
#plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
#for i in range(num_time_steps+1):
#    if autos[i] > 0.01 * autos.max():
#        time = t_eval[i]
#        break
#plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
fig = plt.figure(figsize=(5, 4))
m6_scale = const.diameter_scale**6 * m0_scale
plt.plot(t_eval, m6_scale * moms_1cat_4mom[:,2], 'k',
         label='Total M6')
plt.plot(t_eval, m6_scale * np.sqrt(mom_covs_1cat_4mom[:,2,2]),
         label='1-cat 0-3-6-9')
for check_mom in CHECKED_MOMENTS:
    moments = sorted(FIXED_MOMENTS + [check_mom])
    label_str = '1-cat {}-{}-{}-{}'.format(moments[0], moments[1],
                                           moments[2], moments[3])
    plt.plot(t_eval, m6_scale
             * np.sqrt(mom_covs_1cat_4mom_multimom[check_mom][:,2,2]),
             label=label_str)
ymax = 1.2 * m6_scale * moms_1cat_4mom[:,2].max()
#ymax = 0.1
#for i in range(num_time_steps+1):
#    if autos[i] < accrs[i]:
#        time = t_eval[i]
#        break
#plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
#for i in range(num_time_steps+1):
#    if autos[i] > 0.01 * autos.max():
#        time = t_eval[i]
#        break
#plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')